In [6]:
from ltp import LTP
ltp = LTP()
sents = ltp.sent_split(["他叫汤姆去拿外衣。", "汤姆生病了。他去了医院。"])
segment, _ = ltp.seg(sents)
segment

[['他', '叫', '汤姆', '去', '拿', '外衣', '。'],
 ['汤姆', '生病', '了', '。'],
 ['他', '去', '了', '医院', '。']]

In [10]:
import jieba
seg_list = jieba.cut("我来到北京清华大学")
print("Full Mode: " + " ".join(seg_list))  # 全模式

Full Mode: 我 来到 北京 清华大学
